# Model evaluation and re-training with TransAxx on CIFAR10 dataset

In this notebook you can evaluate different approximate multipliers on various models.
You can also retrain the model for further accuracy improvement

**Note**:
* Currently, the quantization bitwidth supported is 8bit and supported layers are Conv2d and Linear

* Please make sure you have run the installation steps first

* This example notebook approximates Conv2d layers

In [ ]:
from classification.utils import *
device = 'cuda'

## Load dataset

Set your path for the CIFAR10 dataset

'calib dataset' is created from a 10% sample of train data for calibration purposes


In [ ]:
val_data, calib_data = cifar10_data_loader(data_path="/workspace/datasets/cifar10_data", batch_size=128)

## Select a pretrained model

In [ ]:
# an example repo with cifar10 models. you can use your own (ref: https://github.com/chenyaofo/pytorch-cifar-models)
model = torch.hub.load("chenyaofo/pytorch-cifar-models", 'cifar10_repvgg_a0', pretrained=True).to(device)

## Optional: Evaluate default model


In [ ]:
top1 = evaluate_cifar10(model, val_data, device = device)

## Initialize model with axx layers


In [ ]:
# get conv2d layers to approximate
conv2d_layers = [(name, module) for name, module in model.named_modules() if (isinstance(module, torch.nn.Conv2d) or isinstance(module, AdaptConv2D)) and ("head" not in name and "reduction" not in name)]

In [ ]:
len(conv2d_layers)

In [ ]:
# Initialize model with all required approximate multipliers for axx layers. 
# No explicit assignment needed; this step JIT compiles all upcoming multipliers

axx_list = [{'axx_mult' : 'mul8s_acc', 'axx_power' : 1.0, 'quant_bits' : 8, 'fake_quant' : False}]*len(conv2d_layers)
axx_list[3:4] = [{'axx_mult' : 'mul8s_1L2H', 'axx_power' : 0.7082, 'quant_bits' : 8, 'fake_quant' : False}] * 1

start = time.time()
replace_conv_layers(model,  AdaptConv2D, axx_list, 0, 0, layer_count=[0], returned_power = [0], initialize = True)  
print('Time to compile cuda extensions: ', time.time()-start)

In [ ]:
# measure flops of model and compute 'flops' in every layer

import io
from classification.ptflops import get_model_complexity_info
from classification.ptflops.pytorch_ops import linear_flops_counter_hook
from classification.ptflops.pytorch_ops import conv_flops_counter_hook

#hook our custom axx_layers in the appropriate flop counters, i.e. AdaptConv2D : conv_flops_counter_hook
with torch.cuda.device(0):
    total_macs, total_params, layer_specs = get_model_complexity_info(model, (3, 32, 32),as_strings=False, print_per_layer_stat=True,
                                                          custom_modules_hooks={AdaptConv2D : conv_flops_counter_hook}, 
                                                          param_units='M', flops_units='MMac',
                                                          verbose=True)

print(f'Computational complexity:  {total_macs/1000000:.2f} MMacs')
print(f'Number of parameters::  {total_params/1000000:.2f} MParams')


## Run model calibration for quantization

Calibrates the quantization parameters 

Need to re-run it each time the initial model changes

In [ ]:
with torch.no_grad():
    stats = collect_stats(model, calib_data, num_batches=2, device=device)
    amax = compute_amax(model, method="percentile", percentile=99.99, device=device)
    
    # optional - test different calibration methods
    #amax = compute_amax(model, method="mse")
    #amax = compute_amax(model, method="entropy")

## Run model evaluation


In [ ]:
# set desired approximate multiplier in each layer

#at first, set all layers to have the 8-bit accurate multiplier
axx_list = [{'axx_mult' : 'mul8s_acc', 'axx_power' : 1.0, 'quant_bits' : 8, 'fake_quant' : False}]*len(conv2d_layers)

# For example, set the first 10 layers to be approximated with a specific multiplier 
axx_list[0:10] = [{'axx_mult' : 'mul8s_1L2H', 'axx_power' : 0.7082, 'quant_bits' : 8, 'fake_quant' : False}] * 10

returned_power = [0]
replace_conv_layers(model,  AdaptConv2D, axx_list, total_macs, total_params, layer_count=[0], returned_power = returned_power, initialize = False)  
print('Power of approximated operations: ', round(returned_power[0], 2), '%')
print('Model compiled. Running evaluation')

# Run evaluation on the validation dataset
top1 = evaluate_cifar10(model, val_data, device = device)

## Run model retraining


In [ ]:
from classification.train import train_one_epoch

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001) # set desired learning rate
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

#one epoch retrain
train_one_epoch(model, criterion, optimizer, calib_data, device, 0, 10)

## Re-run model evaluation

In [ ]:
top1 = evaluate_cifar10(model, val_data, device = device)